### Data Analysis Questions
---
    1. How many customers has Foodie-Fi ever had?

In [1]:
SELECT COUNT(distinct customer_id) as number_of_customers
FROM subscriptions

(1 row affected)

Total execution time: 00:00:00.019

number_of_customers
1000


    2. What is the monthly distribution of `trial` plan `start_date` values for our dataset - use the start of the month as the group by value

In [2]:
SELECT
    MONTH(start_date) AS month,
    YEAR(start_date) AS year,
    COUNT(customer_id) as number_of_trial
FROM subscriptions s
JOIN plans p
ON p.plan_id = s.plan_id
WHERE p.plan_name = 'trial'
GROUP BY
    MONTH(start_date),
    YEAR(start_date);

(12 rows affected)

Total execution time: 00:00:00.026

month,year,number_of_trial
1,2020,88
2,2020,68
3,2020,94
4,2020,81
5,2020,88
6,2020,79
7,2020,89
8,2020,88
9,2020,87
10,2020,79


    3. What plan `start_date` values occur after the year 2020 for our dataset? Show the breakdown by count of events for each `plan_name`

In [3]:
SELECT 
    s.plan_id,
    p.plan_name,
    COUNT(s.customer_id) as number_of_customers
FROM subscriptions s
JOIN plans p 
ON p.plan_id = s.plan_id
WHERE YEAR(start_date) > 2020
GROUP BY s.plan_id, p.plan_name

(4 rows affected)

Total execution time: 00:00:00.024

plan_id,plan_name,number_of_customers
1,basic monthly,8
4,churn,71
3,pro annual,63
2,pro monthly,60


    4. What is the customer count and percentage of customers who have churned rounded to 1 decimal place?

In [4]:
SELECT
    COUNT(distinct customer_id) as total_customers,
    SUM(case when plan_id = 4 then 1 else 0 END) as churned_customers,
    ROUND(cast(SUM(case when plan_id = 4 then 1 else 0 END) as float)/cast(COUNT(distinct customer_id)as float)*100,1) as percentage_churn
FROM subscriptions;

(1 row affected)

Total execution time: 00:00:00.016

total_customers,churned_customers,percentage_churn
1000,307,30.7


    5. How many customers have churned straight after their initial free trial - what percentage is this rounded to the nearest whole number?

In [5]:
WITH tbl as(
    SELECT *,
        LEAD(plan_id,1) OVER(partition by customer_id order by plan_id) as next_plan 
    FROM subscriptions
)
SELECT 
    COUNT(distinct customer_id) as total_customers,
    SUM(case when (next_plan = 4 and plan_id = 0) then 1 else 0 END) as straigh_churn_after_free_trial,
    ROUND(cast(SUM(case when (next_plan = 4 and plan_id = 0) then 1 else 0 END) as float)/cast(COUNT(distinct customer_id)as float)*100,1) as percentage_straight_churn_after_free_trial
FROM tbl;

(1 row affected)

Total execution time: 00:00:00.062

total_customers,straigh_churn_after_free_trial,percentage_straight_churn_after_free_trial
1000,92,9.2


    6. What is the number and percentage of customer plans after their initial free trial?

In [6]:
WITH tbl as(
    SELECT *,
        LEAD(plan_id,1) OVER(partition by customer_id order by plan_id) as next_plan 
    FROM subscriptions
)
SELECT 
    p.plan_name,
    COUNT(*) as num_each_plans_after_trial,
    ROUND(cast(COUNT(*) as float)/(select COUNT(distinct customer_id) from subscriptions)*100,1) as percentage_each_plans_after_trial
FROM tbl
JOIN plans p ON p.plan_id = tbl.next_plan
WHERE tbl.plan_id = 0
GROUP BY p.plan_name;

(4 rows affected)

Total execution time: 00:00:00.024

plan_name,num_each_plans_after_trial,percentage_each_plans_after_trial
basic monthly,546,54.6
churn,92,9.2
pro annual,37,3.7
pro monthly,325,32.5


    7. What is the customer count and percentage breakdown of all 5 plan_name values at 2020-12-31?

In [7]:
WITH tbl AS(
    SELECT *, 
        LEAD(start_date,1) OVER( PARTITION BY customer_id ORDER BY plan_id) As next_date
    FROM subscriptions
    WHERE start_date <= '2020-12-31'
) 
SELECT
    p.plan_id,
    p.plan_name,
    COUNT(*) as num_each_plans,
    ROUND(cast(COUNT(*) as float)/(select COUNT(distinct customer_id) from subscriptions)*100,1) as percentage_each_plans
FROM tbl
JOIN plans p ON p.plan_id = tbl.plan_id
WHERE next_date is null or next_date >'2020-12-31'
GROUP BY p.plan_id, plan_name
ORDER BY plan_id;

(5 rows affected)

Total execution time: 00:00:00.028

plan_id,plan_name,num_each_plans,percentage_each_plans
0,trial,19,1.9
1,basic monthly,224,22.4
2,pro monthly,326,32.6
3,pro annual,195,19.5
4,churn,236,23.6


    8. How many customers have upgraded to an annual plan in 2020?

In [8]:
SELECT 
    p.plan_name,
    COUNT(*) as num_of_annual_plan
FROM subscriptions s
JOIN plans p ON p.plan_id = s.plan_id 
WHERE p.plan_id = 3 AND YEAR(start_date) = 2020
GROUP BY p.plan_name;

(1 row affected)

Total execution time: 00:00:00.016

plan_name,num_of_annual_plan
pro annual,195


    9. How many days on average does it take for a customer to an annual plan from the day they join Foodie-Fi?

In [9]:
WITH START_CTE AS (
    SELECT 
        customer_id,
        start_date 
    FROM subscriptions s
    INNER JOIN plans p ON s.plan_id = p.plan_id
    WHERE plan_name = 'trial' 
),
ANNUAL_CTE AS (
    SELECT 
        customer_id,
        start_date as start_annual
    FROM subscriptions s
    INNER JOIN plans p ON s.plan_id = p.plan_id
    WHERE plan_name = 'pro annual' 
)
SELECT 
    AVG(DATEDIFF(day, start_date, start_annual)) as average_day
FROM START_CTE s
JOIN ANNUAL_CTE a on a.customer_id = s.customer_id; 

(1 row affected)

Total execution time: 00:00:00.016

average_day
104


    10. Can you further breakdown this average value into 30 day periods (i.e. 0-30 days, 31-60 days etc)

In [10]:
WITH START_CTE AS (   
                SELECT customer_id,
                       start_date 
                FROM subscriptions s
                INNER JOIN plans p ON s.plan_id = p.plan_id
                WHERE plan_name = 'trial' ),

ANNUAL_CTE AS ( SELECT customer_id,
                       start_date as start_annual
                FROM subscriptions s
                INNER JOIN plans p ON s.plan_id = p.plan_id
                WHERE plan_name = 'pro annual' ),

DIFF_DAY_CTE AS (SELECT DATEDIFF(day,start_date,start_annual) as diff_day
                FROM ANNUAL_CTE C2
                LEFT JOIN START_CTE C1 ON C2.customer_id =C1.customer_id),

GROUP_DAY_CTE AS (SELECT*, FLOOR(diff_day/30) as group_day
                FROM DIFF_DAY_CTE)
SELECT CONCAT((group_day *30) +1 , '-',(group_day +1)*30, ' days') as days,
        COUNT(group_day) as number_of_customers
FROM GROUP_DAY_CTE
GROUP BY group_day;

(12 rows affected)

Total execution time: 00:00:00.025

days,number_of_customers
1-30 days,48
31-60 days,25
61-90 days,33
91-120 days,35
121-150 days,43
151-180 days,35
181-210 days,27
211-240 days,4
241-270 days,5
271-300 days,1


    11. How many customers downgraded from a pro monthly to a basic monthly plan in 2020?

In [11]:
WITH tbl as(
    SELECT *,
        LEAD(plan_id,1) OVER(partition by customer_id order by plan_id) as next_plan,
        LEAD(start_date,1) OVER( PARTITION BY customer_id ORDER BY plan_id) As next_date 
    FROM subscriptions
)
SELECT COUNT(*) as num_downgrade
FROM tbl
WHERE plan_id = 2 and next_plan = 1 and YEAR(next_date) = 2020

(1 row affected)

Total execution time: 00:00:00.018

num_downgrade
0
